In [20]:
import tensorflow as tf
import os.path
import time
import math

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('max_steps', 2000, 'Number of steps to run trainer.')
flags.DEFINE_integer('hidden1', 128, 'Number of units in hidden layer 1.')
flags.DEFINE_integer('hidden2', 32, 'Number of units in hidden layer 2.')
flags.DEFINE_integer('batch_size', 100, 'Batch size.  '
                     'Must divide evenly into the dataset sizes.')
flags.DEFINE_string('train_dir', 'data', 'Directory to put the training data.')
flags.DEFINE_boolean('fake_data', False, 'If true, uses fake data '
'for unit testing.')

In [8]:
NUM_CLASSES=10

IMAGE_SIZE=28
IMAGE_PIXELS=IMAGE_SIZE*IMAGE_SIZE

In [18]:
def placeholder_inputs(batch_size):
    images_placeholder=tf.placeholder(tf.float32,shape=(batch_size,IMAGE_PIXELS))
    labels_placeholder=tf.placeholder(tf.int32,shape=(batch_size))
    return images_placeholder, labels_placeholder

In [9]:
def inference(images,hidden1_units,hidden2_units):
    with tf.name_scope('hidden1'):
        weights=tf.Variable(
            tf.truncated_normal([IMAGE_PIXELS,hidden1_units],
                                                stddev=1.0/math.sqrt(float(IMAGE_PIXELS))),
                            name='weights')
        biases=tf.Variable(tf.zeros([hidden1_units]),name='biases')
        hidden1=tf.nn.relu(tf.matmul(images,weights)+biases)
        
    with tf.name_scope('hidden2'):
        weights=tf.Variable(
            tf.truncated_normal([hidden1_units,hidden2_units],
                                               stddev=1.0/math.sqrt(float(hidden1_units))),
                           name='weights')
        biases=tf.Variable(tf.zeros([hidden2_units]),
                          name='biases')
        hidden2=tf.nn.relu(tf.matmul(hidden1,weights)+biases)
        
    with tf.name_scope('softmax_linear'):
        weights=tf.Variable(
            tf.truncated_normal([hidden2_units,NUM_CLASSES],
                               stddev=1.0/math.sqrt(float(hidden2_units))),
        name='weights')
        biases=tf.Variable(tf.zeros([NUM_CLASSES]),
                          name='biases')
        logits=tf.matmul(hidden2,weights)+biases
    return logits

In [23]:
def func_loss(logits,labels):
    # http://colah.github.io/posts/2015-09-Visual-Information/
    # cross-entropy explanation
    labels=tf.to_int64(labels)
    cross_entropy=tf.nn.sparse_softmax_cross_entropy_with_logits(logits,labels,name='xentropy')
    loss=tf.reduce_mean(cross_entropy,name='xentropy_mean')
    return loss

In [12]:
def training(loss,learning_rate):
    tf.scalar_summary(loss.op.name,loss)
    optimizer=tf.train.GradientDescentOptimizer(learning_rate)
    global_step=tf.Variable(0,name='global_step',trainable=False)
    train_op=optimizer.minimize(loss,global_step=global_step)
    return train_op

In [13]:
def evaluation(logits,labels):
    correct=tf.nn.in_top_k(logits,labels,1)
    return tf.reduce_sum(tf.cast(correct,tf.int32))

In [6]:
data_sets=input_data.read_data_sets(FLAGS.train_dir,FLAGS.fake_data)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting data/t10k-labels-idx1-ubyte.gz


In [14]:
def fill_feed_dict(data_set,images_pl,labels_pl):
    images_feed,labels_feed=data_set.next_batch(FLAGS.batch_size,FLAGS.fake_data)
    
    feed_dict={
        images_pl: images_feed,
        labels_pl: labels_feed,
    }
    return feed_dict

In [15]:
def do_eval(sess,
            eval_correct,
            images_placeholder,
            labels_placeholder,
            data_set):
    true_count=0
    steps_per_epoch=data_set.num_examples // FLAGS.batch_size
    num_examples=steps_per_epoch * FLAGS.batch_size
    for step in xrange(steps_per_epoch):
        feed_dict=fill_feed_dict(data_set,
                                images_placeholder,
                                labels_placeholder)
        true_count+=sess.run(eval_correct,feed_dict=feed_dict)
    precision=true_count/num_examples
    print('  Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %(num_examples, true_count, precision))

In [26]:
with tf.Graph().as_default():
    images_placeholder,labels_placeholder=placeholder_inputs(FLAGS.batch_size)
    logits=inference(images_placeholder,FLAGS.hidden1,FLAGS.hidden2)
    loss=func_loss(logits,labels_placeholder)
    train_op=training(loss,FLAGS.learning_rate)
    eval_correct=evaluation(logits,labels_placeholder)
    
    summary=tf.merge_all_summaries()
    init=tf.initialize_all_variables()
    saver=tf.train.Saver()
    
    sess=tf.Session()
    summary_writer=tf.train.SummaryWriter(FLAGS.train_dir,sess.graph)
    sess.run(init)
    
    for step in xrange(FLAGS.max_steps):
        start_time=time.time()
        
        feed_dict=fill_feed_dict(data_sets.train,
                                images_placeholder,
                                labels_placeholder)
        
        _, loss_value=sess.run([train_op,loss],
                              feed_dict=feed_dict)
        duration=time.time()-start_time
        
        if step%100==0:
            print('Step %d: loss = %.2f (%.3f sec)'%(step,loss_value,duration))
            summary_str=sess.run(summary,feed_dict=feed_dict)
            summary_writer.add_summary(summary_str,step)
            summary_writer.flush()
            
        if (step+1) %1000==0 or (step+1)==FLAGS.max_steps:
            checkpoint_file=os.path.join(FLAGS.train_dir,'checkpoint')
            saver.save(sess,checkpoint_file,global_step=step)
            print('Training Data Eval:')
            do_eval(sess,
                   eval_correct,
                   images_placeholder,
                   labels_placeholder,
                   data_sets.train)
            print('Validation Data Eval:')
            do_eval(sess,
                    eval_correct,
                    images_placeholder,
                    labels_placeholder,
                    data_sets.validation)
            print('Test Data Eval:')
            do_eval(sess,
                    eval_correct,
                    images_placeholder,
                    labels_placeholder,
                    data_sets.test)
    sess.close()

Step 0: loss = 2.31 (0.006 sec)
Step 100: loss = 2.15 (0.001 sec)
Step 200: loss = 1.90 (0.001 sec)
Step 300: loss = 1.61 (0.028 sec)
Step 400: loss = 1.07 (0.001 sec)
Step 500: loss = 0.96 (0.001 sec)
Step 600: loss = 0.83 (0.001 sec)
Step 700: loss = 0.88 (0.001 sec)
Step 800: loss = 0.56 (0.001 sec)
Step 900: loss = 0.56 (0.001 sec)
Training Data Eval:
  Num examples: 55000  Num correct: 47233  Precision @ 1: 0.0000
Validation Data Eval:
  Num examples: 5000  Num correct: 4338  Precision @ 1: 0.0000
Test Data Eval:
  Num examples: 10000  Num correct: 8654  Precision @ 1: 0.0000
Step 1000: loss = 0.48 (0.001 sec)
Step 1100: loss = 0.57 (0.001 sec)
Step 1200: loss = 0.38 (0.001 sec)
Step 1300: loss = 0.51 (0.003 sec)
Step 1400: loss = 0.35 (0.032 sec)
Step 1500: loss = 0.33 (0.001 sec)
Step 1600: loss = 0.35 (0.001 sec)
Step 1700: loss = 0.38 (0.008 sec)
Step 1800: loss = 0.36 (0.001 sec)
Step 1900: loss = 0.35 (0.001 sec)
Training Data Eval:
  Num examples: 55000  Num correct: 49237 